<a href="https://colab.research.google.com/github/Profex04/Connect4/blob/main/CSCE_Connect4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

class Board():
    def __init__(self):
        self.N_OF_ROWS = 6;
        self.N_OF_COLS = 7;
        self.stones = ['O', 'X'];
        self.boardMatrix = np.empty((self.N_OF_ROWS,self.N_OF_COLS), dtype='<U1');
        for i in range(self.N_OF_ROWS):
            for j in range(self.N_OF_COLS):
                self.boardMatrix[i][j] = '_';

    def getBoard(self):
        return self.boardMatrix;

    def setStone(self, move, playerNo):
        self.boardMatrix[move.row][move.col] = self.stones[playerNo%2];

    def copyBoard(self, copyFromBoard):
        for i in range(self.N_OF_ROWS):
            for j in range(self.N_OF_COLS):
                self.boardMatrix[i][j] = copyFromBoard[i][j];

    def printBoard(self):
        print(" 0123456");
        for i in range(self.N_OF_ROWS):
            print(i,end='');
            for j in range(self.N_OF_COLS):
                print(self.boardMatrix[i][j],end='');
            print("\n");

class Move:
    def __init__(self, row, col):
        self.row = row;
        self.col = col;

class Player:
    def __init__(self, isComputer, board, game):
        self.isWinner = False;
        self.isComputer = isComputer;
        self.board = board;
        self.game = game;

    def getMove(self, isComputer, board, game, level, predictions):
        if isComputer == False:
            #print("Please enter your move [row, column]");
            row = predictions.row;
            col = predictions.col;
            move = Move(row, col);
            return move;
        else:
            #BOT VS BOT HARD STRATEGY WILL PLAY AGAINST RANDOM
            if level == 0 or (game.computerVsComputer and game.activePlayer == 1):
                return self.strategyEasy(game);
            else:
                #return self.strategyHard(board, game);
                return self.strategyHardFutureLookAheadRecursive(board, game, Move(-1, -1), 1, game.legalMoves)

    def strategyEasy(self, game):
        while True:
            selection = np.random.randint(len(game.legalMoves)); #there is always 6 moves available until end game
            if not (game.legalMoves[selection].row == -1):
                break
        return game.legalMoves[selection];

    def strategyHard(self, board, game):
        for windowLen in range(4, 1):
            if move.row != -1: #n/a spot
                for move in game.legalMoves:
                    if game.checkHorizontal(board, move, windowLen, True) or game.checkVertical(board, move, windowLen, True) or game.checkDiagonal(board, move, windowLen, True, False) or game.checkDiagonal(board, move, windowLen, True, True):
                        return move;
        return self.strategyEasy(game); #cannot find good move

    def strategyHardFutureLookAheadRecursive(self, board, game, prevMove, lookAhead, legalMoves):
        copyBoard = Board()
        copyBoard.copyBoard(board.getBoard());
        copyLegalMoves = np.empty(board.N_OF_COLS, dtype=Move)
        i = 0
        for move in legalMoves:
            copyLegalMoves[i] = move;
            i += 1

        lookAhead += 1; #counter for stopping after looking ahead 3 moves (3 is hardcoded as seen below)

        for windowLen in range(4, 1, -1):
            for counterMove in copyLegalMoves:
                if counterMove.row != -1 and not(counterMove.row == prevMove.row and counterMove.col == prevMove.col): # n/a spot
                    if(game.checkHorizontal(copyBoard, counterMove, windowLen, True) or game.checkVertical(copyBoard, counterMove, windowLen, True) or game.checkDiagonal(copyBoard, counterMove, windowLen, True, False) or game.checkDiagonal(copyBoard, counterMove, windowLen, True, True)):
                        if(lookAhead == 3):
                            return prevMove;

                        #Sets the stone and updates legal moves left on 'pretend' copies of board matrix and legal moves list
                        copyBoard.setStone(counterMove, (game.activePlayer+1)%2);
                        if(counterMove.row != 0):
                            copyLegalMoves[counterMove.col] = Move(counterMove.row-1, counterMove.col);
                        else:
                            copyLegalMoves[counterMove.col] = Move(-1, -1);

                        return self.strategyHardFutureLookAheadRecursive(copyBoard, game, counterMove, lookAhead, copyLegalMoves);
        if(prevMove.row != -1 and prevMove.col != -1):
            return prevMove; #recursive func starts with dummy previous move (-1, -1)
        else:
            return self.strategyEasy(game); #cannot find good move

class Game():
    def __init__(self, p2p, computerVsComputer, level):
        self.board = Board();
        self.p2p = p2p;
        self.level = level;
        self.winner = -1;
        self.gameOver = False;
        self.activePlayer = 0;
        self.computerVsComputer = computerVsComputer
        self.players = np.empty(2, dtype=Player)
        if not computerVsComputer:
            self.players[0] = Player(False, self.board, self);
            isComputer = False if p2p else True;
            self.players[1] = Player(isComputer, self.board, self);
        else:
            self.players[0] = Player(True, self.board, self)
            self.players[1] = Player(True, self.board, self)
        self.legalMoves = np.empty(self.board.N_OF_COLS)
        self.legalMoves = [Move(5,0), Move(5,1), Move(5,2), Move(5,3), Move(5,4), Move(5,5), Move(5,6)];
        #self.runGameLoop();
        self.playerOneWins = False;
        self.playerTwoWins = False;

    def getWinner(self):
        return self.winner;

    def getActivePlayer(self):
        return self.activePlayer;

    def setLevel(self, level):
        self.level = level;

    def setP2P(self, p2p):
        self.p2p = p2p;

    def getGameOver(self):
        return self.gameOver;

    def gameTurn(self, activePlayer, predictions):
        isComputer = False if ((self.p2p or activePlayer % 2 == 0) and not self.computerVsComputer) else True
        move = self.players[activePlayer].getMove(isComputer, self.board, self, self.level, predictions)
        while not self.checkLegalMove(self.board, move, activePlayer):
            move = self.players[activePlayer].getMove(isComputer, self.board, self, self.level, predictions)
        self.board.setStone(move, activePlayer)
        if move.row != 0:
            self.legalMoves[move.col] = Move(move.row - 1, move.col)
        else:
            self.legalMoves[move.col] = Move(-1, -1)  # spot not available anymore
        return move


    def checkHorizontal(self, board, move, windowLen, testFlag):
        horizontalCheck = False;
        row = move.row; #only check the row of the last move
        for j in range(max(0, move.col-(windowLen)), min(move.col+1, board.N_OF_COLS-windowLen+1)):
            if j+windowLen <= board.N_OF_COLS:
                consecutiveStones = board.getBoard()[row, j:j+windowLen]
                if len(consecutiveStones) == windowLen:
                    horizontalCheck = self.checkIfConnected(consecutiveStones, windowLen, testFlag);
                if horizontalCheck == True:
                    return horizontalCheck;
        return horizontalCheck;

    def checkVertical(self, board, move, windowLen, testFlag):
        verticalCheck = False;
        col = move.col; #only check the col of the last move
        for i in range(max(0, move.row-(windowLen)), min(move.row+1, board.N_OF_ROWS-windowLen+1)):
            consecutiveStones = board.getBoard()[i:i+windowLen, col]
            if len(consecutiveStones) == windowLen:
                verticalCheck = self.checkIfConnected(consecutiveStones, windowLen, testFlag);
            if verticalCheck == True:
                return verticalCheck;
            consecutiveStones = [];
        return verticalCheck;

    def checkDiagonal(self, board, move, windowLen, testFlag, reverseFlag):
        diagonalCheck = False
        consecutiveStones = []

        if abs(move.row-move.col) <= 3 and not reverseFlag:
            for i, j in zip(range(max(0, move.row - (windowLen - 1)), min(move.row, board.N_OF_ROWS - windowLen) + 1), range(max(0, move.col - (windowLen - 1)), min(move.col, board.N_OF_COLS - windowLen) + 1)):
                for k in range(windowLen):
                    consecutiveStones.append(board.getBoard()[i + k][j + k])
                    if i + k + 1 >= board.N_OF_ROWS or j + k + 1 >= board.N_OF_COLS:
                        break
                if len(consecutiveStones) == windowLen:
                     diagonalCheck = self.checkIfConnected(consecutiveStones, windowLen, testFlag)
                if diagonalCheck:
                     return diagonalCheck
                consecutiveStones = []

        if move.row + move.col <= 8 and move.row + move.col > 2 and reverseFlag:
            i_start = max(0, move.row - (windowLen - 1))
            j_start = min(move.col + (move.row - i_start), board.N_OF_COLS)
            for i, j in zip(range(i_start, min(move.row, board.N_OF_ROWS - windowLen) + 1), range(j_start, move.col + 1)):
                for k in range(windowLen):
                    consecutiveStones.append(board.getBoard()[i + k][j - k])
                    if i + k + 1 >= board.N_OF_ROWS or j - k - 1 < 0:
                        break
                if len(consecutiveStones) == windowLen:
                    diagonalCheck = self.checkIfConnected(consecutiveStones, windowLen, testFlag)
                if diagonalCheck:
                    return diagonalCheck
                consecutiveStones = []
        return diagonalCheck;

    def checkLegalMove(self, board, move, activePlayer):
        while move.row < board.N_OF_ROWS - 1 and board.getBoard()[move.row + 1][move.col] == '_':
            move.row += 1

        if board.getBoard()[move.row][move.col] != '_' or move.col >= board.N_OF_COLS:
            return False
        return True

    def checkIfConnected(self, consecutiveStones, windowLen, testFlag):
        left = 0
        sum = 0
        prevSum = 0
        while left < windowLen and not (np.abs(sum) < np.abs(prevSum)):
            prevSum = sum
            if consecutiveStones[left] == 'X':
                sum += 1
            elif consecutiveStones[left] == 'O':
                sum -= 1
            left += 1

        if testFlag:
            blanks = 0
            for elem in consecutiveStones:
                if elem == '_':
                    blanks += 1
            if blanks == 1 and (sum == windowLen - 1 or sum == -1 * (windowLen - 1)):
                return True

        if sum == windowLen or sum == -1 * windowLen:
            return True
        else:
            return False

    def checkIfWin(self, move):
        if self.checkHorizontal(self.board, move, 4, False) == True:
            self.winner = self.activePlayer;
            self.gameOver = True;
        elif self.checkVertical(self.board, move, 4, False) == True:
            self.winner = self.activePlayer;
            self.gameOver = True;
        elif self.checkDiagonal(self.board, move, 4, False, False) == True:
            self.winner = self.activePlayer;
            self.gameOver = True;
        elif self.checkDiagonal(self.board, move, 4, False, True) == True:
            self.winner = self.activePlayer;
            self.gameOver = True;
        self.playerOneWins = self.gameOver and self.activePlayer==0;
        self.playerTwoWins = self.gameOver and self.activePlayer==1;
        return self.gameOver;

class Environment:
    def __init__(self, computerVsComputer, level):
        self.game = Game(False, computerVsComputer, level); #p2p flag=false, 1=hard level 0=easy level
        self.actionSpace = np.arange(len(self.game.legalMoves))
        self.observationSpace = self.encode(self.game.board.boardMatrix)
        self.computerVsComputer = computerVsComputer
        self.level = level

    def encode(self, stateMatrix):
        return tuple(tuple(row) for row in stateMatrix)

    def reset(self):
        self.game = Game(False, self.computerVsComputer, self.level);
        self.observationSpace = self.encode(self.game.board.boardMatrix)
        state = self.observationSpace
        return state

    def step(self, action):
        move = self.game.gameTurn(self.game.activePlayer, action);
        self.game.checkIfWin(move)
        self.observationSpace = self.encode(self.game.board.boardMatrix)
        if self.game.playerOneWins:
            reward = 20
        elif self.game.playerTwoWins:
            reward = -10
        else:
            reward = -1
        done = self.game.gameOver
        self.game.activePlayer = (self.game.activePlayer+1) % 2;
        return (self.observationSpace, reward, done)

    def sample(self):
        while True:
            selection = np.random.randint(len(self.game.legalMoves)); #there is always 6 moves available until end game
            if not (self.game.legalMoves[selection].row == -1):
                break
        #return [self.game.legalMoves[selection].row, self.game.legalMoves[selection].col];
        return selection

    def render(self):
        #self.game.board.printBoard();
        renderedObservationSpace = np.empty((self.game.board.N_OF_ROWS, self.game.board.N_OF_COLS), dtype='<U1')
        for row in range(np.shape(self.observationSpace)[0]):
            for col in range(np.shape(self.observationSpace)[1]):
                renderedObservationSpace[row][col] = self.observationSpace[row][col]
        return renderedObservationSpace

#game = Game(False, 1); #p2p flag=false, 1=hard level 0=easy level

from IPython.display import clear_output
from time import sleep
def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.15)



In [ ]:

#BOT VS BOT (WITHOUT REINFORCEMENT LEARNING)
env = Environment(True, 1) #computerVsComputer flag=True, level=hard for Player 1, easy for Player 2
total_epochs, total_won = 0, 0
episodes = 100
for i in range(1, episodes):
    state = env.reset()

    done = False
    won = 0
    epochs = 0
    reward = 0
    frames = [] # for animation
    while not done:
        no_moves_left = all(move.row == -1 for move in env.game.legalMoves)
        if(no_moves_left): break

        actionIndex = env.sample()
        action = env.game.legalMoves[actionIndex]
        state, reward, done = env.step(action)
        frames.append({
            'frame': env.render(),
            'state': state,
            'action': (action.row, action.col),
            'reward': reward
            })
        epochs += 1
        if reward == 20:
            won = 1
        else:
            won = 0
    print("Timesteps taken: {}".format(epochs))
    print_frames(frames)
    total_epochs += epochs
    total_won += won

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Number of times won: {total_won / episodes}")



[['X' '_' '_' '_' '_' '_' '_']
 ['X' '_' '_' '_' '_' '_' '_']
 ['O' '_' '_' '_' '_' '_' '_']
 ['X' 'O' 'O' '_' '_' '_' '_']
 ['O' 'X' 'X' 'O' 'X' '_' '_']
 ['O' 'O' 'X' 'O' 'O' 'X' 'X']]
Timestep: 18
State: (('X', '_', '_', '_', '_', '_', '_'), ('X', '_', '_', '_', '_', '_', '_'), ('O', '_', '_', '_', '_', '_', '_'), ('X', 'O', 'O', '_', '_', '_', '_'), ('O', 'X', 'X', 'O', 'X', '_', '_'), ('O', 'O', 'X', 'O', 'O', 'X', 'X'))
Action: (4, 4)
Reward: -1


In [ ]:
import time
#IMPLEMENTING Q LEARNING
env = Environment(False, 0) #during training, player 1 will be AI, player 2 will be computer bot (hard strategy)
q_table = {} #although there are possible 3^42 possible states theoretically, due to game rules this q table will be much much smaller and dynamically managed

start_time = time.time()

#Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalty, reward, = 0, 0, 0
    #frames = [] # for animation
    done = False

    while not done:

        no_moves_left = all(move.row == -1 for move in env.game.legalMoves)
        if(no_moves_left): break

        if state not in q_table:
            q_table[state] = np.zeros(len(env.actionSpace))

        if np.random.uniform(0, 1) < epsilon:
            actionIndex = env.sample() # Explore action space
        else:
            actionIndex = np.argmax(q_table[state]) # Exploit learned values
            if (env.game.legalMoves[actionIndex].row == -1):
                penalty = -1 #penalty for choosing a non-legal move
                actionIndex = env.sample()
            else:
                penalty = 0

        action = env.game.legalMoves[actionIndex]
        next_state, reward, done = env.step(action)
        reward += penalty

        if next_state not in q_table:
            q_table[next_state] = np.zeros(len(env.actionSpace))

        old_value = q_table[state][actionIndex]
        next_max = np.max(q_table[next_state])

        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state][actionIndex] = new_value

        state = next_state
        #frames.append({
        #'frame': env.render(),
        #'state': state,
        #'action': (action.row, action.col),
        #'reward': reward
        #})
        epochs += 1
        #env.game.board.printBoard()
        #clear_output(wait=True)

    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

    #print_frames(frames)

print("Training finished.\n")
print("--- %s seconds ---" % (time.time() - start_time))


Episode: 100000
Training finished.

--- 286.27652502059937 seconds ---


In [ ]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_won = 0, 0
episodes = 100
env.level = 0 #AI WILL PLAY AGAINST RANDOM PLAYING BOT FOR COMPARISON

for episode in range(episodes):
    state = env.reset()
    epochs, penalty, reward = 0, 0, 0
    frames = [] #for animation
    done = False

    while not done:
        no_moves_left = all(move.row == -1 for move in env.game.legalMoves)
        if(no_moves_left): break
        if state not in q_table:
            q_table[state] = np.zeros(len(env.actionSpace))
        actionIndex = np.argmax(q_table[state])
        if (env.game.legalMoves[actionIndex].row == -1):
            actionIndex = env.sample()
            penalty = -1
        else:
            penalty = 0
        action = env.game.legalMoves[actionIndex]
        state, reward, done = env.step(action)
        reward += penalty

        if reward == 20:
            won = 1
        else:
            won = 0

        frames.append({
        'frame': env.render(),
        'state': state,
        'action': (action.row, action.col),
        'reward': reward
        })
        epochs += 1


    total_epochs += epochs
    total_won += won
    print_frames(frames)

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Percentage won: {total_won / episodes}")


[['_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' 'X' '_' '_' '_']
 ['O' 'O' 'O' 'O' 'X' '_' 'X']]
Timestep: 7
State: (('_', '_', '_', '_', '_', '_', '_'), ('_', '_', '_', '_', '_', '_', '_'), ('_', '_', '_', '_', '_', '_', '_'), ('_', '_', '_', '_', '_', '_', '_'), ('_', '_', '_', 'X', '_', '_', '_'), ('O', 'O', 'O', 'O', 'X', '_', 'X'))
Action: (5, 0)
Reward: 20
Results after 100 episodes:
Average timesteps per episode: 9.36
Percentage won: 0.91
